# Clase 3: Ilustración del Teorema del Límite Central 

Prof. Tito Homem-de-Mello

## Tomaremos como ejemplo 100 lanzamientos de un dado de 6 caras

### Primero, carguemos algunas bibliotecas necesarias

In [ ]:
#install.packages('tidyr')
#install.packages('ggplot2')

In [ ]:
library(stats)
library(purrr)
library(tidyr)
library(ggplot2)

### Calculemos medias de $n$ lanzamientos, $n=1,\ldots,100$

In [ ]:
mu <- mean(1:6) #Verdadera media
sigma <- sd(1:6) #verdadera desviacion estandar
N <- 100 #Nro. de lanzamientos 
medias <- vector("double",N)  #Inicialización
sN <- vector("double",N) 

u <- rdunif(N,1,6) #Sortea N muestras de una Uniforme(1,2,...6)
medias <- cumsum(u) / 1:N #Calcula media de los primeros n elementos, n=1,...N


### Gráfico de las medias

In [ ]:
medias_df <- as.data.frame(medias)
medias_df$n <- 1:N
ggplot(medias_df, aes(n,medias)) +
        theme(plot.title = element_text(size = 25),
              axis.title.y=element_text(size = 20),
              axis.title.x=element_text(size = 20))+
        geom_line()+
        labs(y="Medias de los primeros n valores",
             title="Gráfico de medias")

### Pero ese es apenas 1 experimento! Repitámoslo más veces

In [ ]:
nrep <- 200 #2,5,50,200
{
  medias_rep <- array(rep(0,nrep*N),dim=c(N,nrep))

  for (r in 1:nrep)
  {
    u <- rdunif(N,1,6)
    medias_rep[,r] <- cumsum(u) / 1:N 
    sN[r] <- sd(u)
  }
    
    medias_df <- as.data.frame(medias_rep)
    medias_df$n <- 1:N
    plotdata <- medias_df %>% tidyr::gather("rep", "media", 1:nrep) 
    ggplot(plotdata, aes(n,media,group=rep,color=rep)) +
        geom_line(show.legend = F)+
        theme(plot.title = element_text(size = 25),
              axis.title.y=element_text(size = 20),
              axis.title.x=element_text(size = 20))+
        labs(y="Medias de los primeros n valores",
             title=paste0("Gráfico de medias para ",nrep," repeticiones"))
}


# 2. Ilustración del Teorema del Límite Central 



### Veamos un histograma (respecto de las repeticiones) de las medias correspondientes a $N$ muestras

In [ ]:
ggplot(subset.data.frame(plotdata,n==N), aes(media)) +
        geom_histogram(aes(y = after_stat(count / sum(count))), binwidth=0.1, color="black", fill="gray")+
        theme(plot.title = element_text(size = 20),
              axis.title.y=element_text(size = 20),
              axis.title.x=element_text(size = 20))+
       labs(x="media",
            y="frecuencia",
            title=paste0("Histograma de las ",nrep," repeticiones con N=",N))
















### Chequeemos la cobertura de los intervalos de confianza

In [ ]:
nrep <- 200 #50
lower<-vector("double",nrep)
upper<-vector("double",nrep)

for (r in 1:nrep)
{
  lower[r]<-medias_rep[N,r]-1.96*sN[r]/sqrt(N)
  upper[r]<-medias_rep[N,r]+1.96*sN[r]/sqrt(N)
}
low <- data.frame(t(lower))
up <- data.frame(t(upper))
names(low) <- 1:nrep
names(up) <- 1:nrep
inter <- rbind(low,up)
inter_table <- inter %>% tidyr::gather("rep", "intervalo", 1:nrep) 

plotfig <- ggplot(mapping=aes(intervalo,rep))+
              geom_line(data=inter_table)+
              theme(axis.text.y=element_blank())+
              theme(plot.title = element_text(size = 18),
                    axis.title.y=element_text(size = 20),
                    axis.title.x=element_text(size = 20))+
              labs(title=paste0("Intervalos de confianza para ",nrep," repeticiones con N=",N))
plotfig



### Seleccionamos intervalos que no cubren mu


In [ ]:
inter_out <- c()
rp <- 1:nrep
outint <- rp[lower>mu | upper<mu]
for (r in 1:length(outint))
{
  inter_out <- rbind(inter_out,subset.data.frame(inter_table,rep==outint[r]))
}

newrow1 <- data.frame(rep=1,intervalo=mu)
newrow2 <- data.frame(rep=nrep,intervalo=mu)
vert <- rbind(newrow1,newrow2)   #linea vertical en mu


plotfig+
  geom_line(data=vert,color="blue")+
  geom_line(data=inter_out,color="red")
#  

### Ilustrando convergencia en distribución

In [ ]:
for (N in c(5,10,20,50,100)){
unorm<-sqrt(N)*(medias_rep[N,]-mu)/sigma
   
title=paste0("Gráfico distribución acumulada para N=",N)
plot(function(p) pnorm(p,mean=0,sd=1),-3,3,main=title,ylab="F_n(x)",col=2)
plot(ecdf(unorm), ylab="Fn(x)", verticals = TRUE,do.points=FALSE,add=T)}